In [4]:
%load_ext autoreload
%autoreload 2

In [13]:
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from xmitgcm import utils
from xmitgcm import open_mdsdataset
from xgcm import Grid
from xhistogram.xarray import histogram
from dask.diagnostics import ProgressBar

In [14]:
import sys
sys.path.append("scripts")
from filesystem import *

### Load and preprocess simulation output

In [15]:
ds, grid = load_RT_canyon_hydrostatic()

In [16]:
release_time = np.datetime64(ds.release_time)
final_time = release_time + np.timedelta64(42,'h')
dye_loc = ds.dye_release_loc

# Only need times between release and end of FCTD surveys
ds = ds.sel(time=slice(release_time - np.timedelta64(1,'h'), final_time))
ds = ds.where(ds['hFacC']!=0.)

# Small chunks to make xhistogram happy
ds = ds.chunk({'Z':10})

### Load FCTD transect coordinates 

In [17]:
import scipy.io as sio
import astropy
from astropy.time import Time

In [18]:
fctd = sio.loadmat("data/FCTD_transects.mat", simplify_cells=True)['transects']

def get_trajectory(tjname):
    try:
        lon = fctd[tjname]['fctd_lon']
        lat = fctd[tjname]['fctd_lat']
        time = fctd[tjname]['time']
    except:
        lon = fctd[tjname]['fish_lon']
        lat = fctd[tjname]['fish_lat']
        time = fctd[tjname]['time']

    t0 = Time(0.0000, format="decimalyear", scale='utc').plot_date - 1.; # shift by one day to convert to matlab datenum format
    time = (Time(t0 + time , format='plot_date', scale='utc').datetime).astype("datetime64")
    
    trajectory = xr.Dataset({
        'XC': xr.DataArray(lon + 1.e-8*np.random.rand(lon.size), dims='cast'),
        'YC': xr.DataArray(lat + 1.e-8*np.random.rand(lon.size), dims='cast'),
        'time': xr.DataArray(time, dims='cast')
    })
    return trajectory

### Subsampling experiments

In [19]:
exp = "realistic"
for k,v in fctd.items():
    if k=="timeseries": continue
    with ProgressBar():
        traj = get_trajectory(k)
        tran = ds.interp(traj).compute()
        tran.to_netcdf(f"data/simulated_transects/{k}_{exp}.nc", format="NETCDF4")
        tran.close()

/home/hdrake/.conda/envs/rockall/lib/python3.10/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "dtf2d" yielded 1 of "dubious year (Note 6)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/home/hdrake/.conda/envs/rockall/lib/python3.10/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "utctai" yielded 1 of "dubious year (Note 3)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/home/hdrake/.conda/envs/rockall/lib/python3.10/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "taiutc" yielded 1 of "dubious year (Note 4)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


[########################################] | 100% Completed | 26.5s


/home/hdrake/.conda/envs/rockall/lib/python3.10/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "dtf2d" yielded 1 of "dubious year (Note 6)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/home/hdrake/.conda/envs/rockall/lib/python3.10/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "utctai" yielded 1 of "dubious year (Note 3)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/home/hdrake/.conda/envs/rockall/lib/python3.10/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "taiutc" yielded 1 of "dubious year (Note 4)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


[########################################] | 100% Completed | 19.7s


/home/hdrake/.conda/envs/rockall/lib/python3.10/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "dtf2d" yielded 1 of "dubious year (Note 6)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/home/hdrake/.conda/envs/rockall/lib/python3.10/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "utctai" yielded 1 of "dubious year (Note 3)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/home/hdrake/.conda/envs/rockall/lib/python3.10/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "taiutc" yielded 1 of "dubious year (Note 4)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


...

[########################################] | 100% Completed | 58.5s


/home/hdrake/.conda/envs/rockall/lib/python3.10/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "dtf2d" yielded 1 of "dubious year (Note 6)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/home/hdrake/.conda/envs/rockall/lib/python3.10/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "utctai" yielded 1 of "dubious year (Note 3)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/home/hdrake/.conda/envs/rockall/lib/python3.10/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "taiutc" yielded 1 of "dubious year (Note 4)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


[########################################] | 100% Completed |  1min  5.4s


/home/hdrake/.conda/envs/rockall/lib/python3.10/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "dtf2d" yielded 1 of "dubious year (Note 6)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/home/hdrake/.conda/envs/rockall/lib/python3.10/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "utctai" yielded 1 of "dubious year (Note 3)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/home/hdrake/.conda/envs/rockall/lib/python3.10/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "taiutc" yielded 1 of "dubious year (Note 4)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


[########################################] | 100% Completed | 39.2s


In [ ]:
exp = "instantaneous-init-time"
for k,v in fctd.items():
    if k=="timeseries": continue
    with ProgressBar():
        traj = get_trajectory(k)
        
        # Sample transects as instantaneous snapshots, rather than time-dependent transects
        tmp = traj['time'].copy()
        traj['time'] = traj['time'].where(False, traj['time'].isel(cast=0))
        tran = ds.interp(traj).compute()
        
        # Override with old times for convenient comparison to base case
        tran['time'] = tran['time'].where(False, tmp)
        tran = tran.assign_coords(
            {'hours_since_release': ((tran['time'] - release_time)/(1.e9)).astype('float64')/3600.}
        )
        tran.to_netcdf(f"data/simulated_transects/{k}_{exp}.nc")
        tran.close()

In [ ]:
exp = "instantaneous-mean-time"
for k,v in fctd.items():
    if k=="timeseries": continue
    with ProgressBar():
        traj = get_trajectory(k)
        
        # Sample transects as instantaneous snapshots, rather than time-dependent transects
        tmp = traj['time'].copy()
        traj['time'] = traj['time'].where(False, traj['time'].mean())
        tran = ds.interp(traj).compute()
        
        # Override with old times for convenient comparison to base case
        tran['time'] = tran['time'].where(False, tmp)
        tran = tran.assign_coords(
            {'hours_since_release': ((tran['time'] - release_time)/(1.e9)).astype('float64')/3600.}
        )
        tran.to_netcdf(f"data/simulated_transects/{k}_{exp}.nc")
        tran.close()